# PCAP Analysis

Sources : 
- https://adriangcoder.medium.com/pandas-tricks-and-tips-a7b87c3748ea
- https://www.stamus-networks.com/blog/jupyter-playbooks-for-suricata-part-3



Certains outils ont été spécialement développés pour analyser le traffic réseau et détecter à travers des règles définies, un comportement malveillant/suspect.


In [ ]:
import json
import pandas as pd
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
import re
from IPython.display import display, HTML

import ipaddress as ip
from msticpy.transform.iocextract import IoCExtract
# Instantiate an IoCExtract object
#from msticpy.transform import IoCExtract
ioc_extractor = IoCExtract()

import msticpy as mp
mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()
ioc_extract = IoCExtract()

#Expand the width of the cells
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def ip_type(string):
    if ip.ip_address(string).is_private:
        return 'Private'
    elif ip.ip_address(string).is_multicast:
        return 'Multicast'
    elif ip.ip_address(string).is_reserved:
        return 'Reserved'
    elif ip.ip_address(string).is_loopback:
        return 'Loopback'
    elif ip.ip_address(string).is_global:
        return 'Public'
    elif ip.ip_address(string).is_link_local:
        return 'Link local'



# Chemin du fichier à analyser
pcapFile = {}
pcapFile_path = "/home/kidrek/Downloads/5H42K.pcap"



## Analyse rapide du PCAP via Tshark

Il est possible de parcourir facilement le contenu d'un fichier PCAP grâce à l'outil : **tshark**, une version console de l'outil **wireshark**.

```
# Read all pcap file 
$ tshark -r {pcap_filepath}

# Read all pcap file without resolve domain name 
$ tshark -nr {pcap_filepath}
```



Affichage des conversations **TCP** présentes au sein du PCAP

In [ ]:
!tshark -nr $pcapFile_path -q -z conv,tcp

Affichage des conversations **UDP** présentes au sein du PCAP

In [ ]:
!tshark -nr $pcapFile_path -q -z conv,udp

Il est possible d'aller plus loin via l'usage de filtres.

* Filter on HTTP method


In [ ]:
!tshark -r $pcapFile_path -Y "http.request.method==GET" | grep -i 'swellheaded.php'

Show all HTTP requests and URLs : \
It could be interesting to identify exe or archive downloads.

In [ ]:
#!tshark -r $pcapFile_path -Y 'ip.src == 1.2.3.4 and http.request.method == "GET"' -T fields -e http.request.method -e http.request.version -e http.request.full_uri | head -n 1
!tshark -nr $pcapFile_path -Y 'http.request.method == "POST" or http.request.method == "GET"' -T fields -e tcp.stream  -e http.request.method -e http.request.version -e http.request.full_uri | egrep -i '.zip|.exe'

Get all data about a specific tcp stream id

In [ ]:
!tshark -nr $pcapFile_path -Y 'tcp.stream == 119' -T fields -e data.data

In [ ]:
!tshark -nr $pcapFile_path -Y 'http.request.method == "POST" or http.request.method == "GET"' -T fields -e tcp.stream  -e http.request.method -e http.request.version -e http.request.full_uri -e data.data

* Show all User-Agent

In [ ]:
!tshark -nr $pcapFile_path -Y "http.user_agent" -Tfields -e ip.addr -e http.user_agent 


source : https://jsur.in/post/2020-02-19-tshark-cheatsheet


## Analyse du PCAP via Suricata

Une fois l'analyse rapide terminée, il est important de faire appel à des moteurs de détection type NIDS, spécialisé Réseau, tel que **Suricata**. 
Avant de débuter l'analyse du pcap, il est indispensable de mettre à jour les règles de détection ```suricata-update```.

In [ ]:
LOGDIR = "/tmp/suricata/logs"
!rm -rf $LOGDIR 2>/dev/null; mkdir -p $LOGDIR

# Update rules
!sudo suricata-update 1>/dev/null

# Start analyse
!suricata -S /var/lib/suricata/rules/suricata.rules -r $pcapFile_path -l $LOGDIR -v 

Si Suricata identifie à travers ses règles des indicateurs de compromissions, alors le nombre d'alerte devrait être supérieur à 0. 

Si tel est le cas, ces alertes sont consultables au sein du fichier eve.json ou fast.log. \
Les étapes suivantes considèrent que des alertes ont été générées. L'action initiale consiste donc à parcourir le fichier **eve.json** et le charger dans un dataFrame pour analyser plus facilement les données qu'il contient.



In [ ]:
# Load nested eve.json in dataFrame
with open(f"{ LOGDIR }/eve.json", "r") as eveFile:
    df_suricata = pd.json_normalize([
        json.loads(line) for line in eveFile
    ], max_level=1)

df_suricata['flow_id'] = df_suricata['flow_id'].fillna(0).astype(int)
df_suricata['alert.signature_id'] = df_suricata['alert.signature_id'].fillna(0).astype(int)
df_suricata['dest_port'] = df_suricata['dest_port'].fillna(0).astype(int)
df_suricata['src_port'] = df_suricata['src_port'].fillna(0).astype(int)

df_suricata['flow.start'] = pd.to_datetime(df_suricata['flow.start'], format='%Y-%m-%d %H:%M:%S')
df_suricata['timestamp'] = pd.to_datetime(df_suricata['timestamp'], format='%Y-%m-%d %H:%M:%S')

### Qualification des alertes recensées par Suricata



In [ ]:
df_suricata_alert = df_suricata[(df_suricata.event_type == "alert") & (df_suricata['alert.category'].str.contains('Not Suspicious') == False)][['alert.severity','alert.signature_id','alert.signature','src_ip','src_port','dest_ip','dest_port','proto','app_proto','flow.start','flow_id','flow.bytes_toserver','flow.bytes_toclient']].sort_values(by=['alert.severity'], ascending=False)
display(df_suricata_alert.head(20))

Une fois les alertes qualifiées, nous allons réaliser une extraction des IOCs potentiellement présents dans le PCAP et les rechercher dans des bases de connaissances de la menace.

In [ ]:
# any IoCs in the string?
iocs_found = ioc_extractor.extract(data=df_suricata.fillna(''), columns=['src_ip','dest_ip','dns.rrname'])
iocs_found = iocs_found['Observable'].drop_duplicates()
df_ti = ti.lookup_iocs(data=iocs_found, providers=["VirusTotal", "OTX"])
df_suspnetworkconnections = df_ti[df_ti['Result']==True]
df_suspnetworkconnections = pd.json_normalize(data=df_suspnetworkconnections[['Ioc','Provider','Details']].to_dict(orient='records')).sort_values(by=['Details.pulse_count'], ascending=False)
df_suspnetworkconnections[['Ioc','Provider','Details.pulse_count','Details.names','Details.references']]

### Statistiques

#### Déterminer les types de connexions réalisés

In [ ]:
df_suricata.groupby(by='event_type').size().reset_index(name='count').sort_values(by=['count'])

#### Determiner les noms de domaines recherchés

Cette analyse permet de recenser les noms de domaines non communs pouvant résulter d'une attaque en cours.

In [ ]:
df_suricata[df_suricata.event_type=='dns'][['timestamp','dns.rrname','dns.rrtype','dns.rcode','dns.grouped']].dropna()

#### Déterminer les fréquences de connexions

Il est également possible d'identifier les récurrences de connexions vers une même adresse ip de destination.

In [ ]:
df_suricata.groupby('dest_ip').size().reset_index(name='count').sort_values(by=['count'], ascending=False)

#### [TODO] Determiner les connexions réalisées à intervale régulier en parcourant l'ensemble des flow.

Cette analyse permet notamment d'identifier les beacons présents sur notre réseau.

In [ ]:
# source : https://towardsdatascience.com/6-visualization-tricks-to-handle-ultra-long-time-series-data-57dad97e0fc2

df_flow = df_suricata[['timestamp','flow_id','src_ip','dest_ip','flow.bytes_toserver','flow.bytes_toclient']].sort_values(by=['timestamp'])

# Generate a graph filtered on specific remote ip
#df_flow = df_suricata[df_suricata.dest_ip == '10.6.2.1' ][['timestamp','flow_id','src_ip','dest_ip','flow.bytes_toserver','flow.bytes_toclient']].sort_values(by=['timestamp'])

display(px.box(df_flow, y='flow.bytes_toclient', x ='timestamp'))
display(px.box(df_flow, y='flow.bytes_toserver', x ='timestamp'))
